In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import utils
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers
import os



Using TensorFlow backend.


In [37]:
import joblib

In [2]:
train_df = pd.read_csv("CSVs/categorical_training.csv")
train_df.head()

,image_path,species
0,agaricus_arvensis/agaricusarvensis1.jpg,agaricus_arvensis
1,agaricus_arvensis/agaricusarvensis3.jpg,agaricus_arvensis
2,agaricus_arvensis/agaricusarvensis4.jpg,agaricus_arvensis
3,agaricus_arvensis/agaricusarvensis5.jpg,agaricus_arvensis
4,agaricus_augustus/agaricusaugustus1.jpg,agaricus_augustus


In [3]:
test_df = pd.read_csv('CSVs/categorical_testing.csv')
test_df.head()

,image_path
0,agaricus_arvensis/agaricusarvensis2.jpg
1,agaricus_augustus/agaricusaugustus2.jpg
2,albatrellus_confluens/albatrellusconfluens2.jpg
3,amanita_fulva/amanitafulva2.jpg
4,amanita_muscaria/amanitamuscaria2.jpg


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 483 entries, 0 to 482
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_path  483 non-null    object
 1   species     483 non-null    object
dtypes: object(2)
memory usage: 7.7+ KB


In [5]:
num_classes =train_df['species'].nunique()
num_classes

138

# Generate Training data

In [6]:
image_height, image_width = 81,81
reg_num = 0.00001
root_folder = "species_classifier_images"

In [7]:
train_df.loc[0][0]

'agaricus_arvensis/agaricusarvensis1.jpg'

In [8]:
# Convert Image data into an array and store into a list
train_image = []
count=0
while count <len(train_df):
    img = image.load_img(f"{root_folder}/train/{train_df.loc[count][0]}", target_size=(image_height,image_width,3), color_mode='rgb')
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
    count+=1
    


In [9]:
# X = image data
X = np.array(train_image)

In [10]:
# Y = class data - must be one hot encoded and transformed into array to match X data
encoder = OneHotEncoder()
encoder.fit(train_df[['species']])

y = encoder.transform(train_df[['species']]).toarray()

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

# Build Neural Network

# Function to use in grid searching and optimizing the neural network

In [20]:
def run_nn(layer_one_neurons=64, reg_num=0.0 , drop_out=.25):
   
    model = Sequential()
    
    #First Input Layer
    model.add(Conv2D(layer_one_neurons,
                 kernel_size= (3,3), 
                 input_shape=(image_height,image_width,3), 
                 activation='relu',
                ))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    #additional CONV layers
    model.add(Conv2D(32,
                 kernel_size= (3,3), 
                 activation='relu',
                 kernel_regularizer=regularizers.l2(reg_num)
                ))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(drop_out))

    model.add(Conv2D(32,
                     kernel_size= (3,3), 
                     activation='relu',
                     kernel_regularizer=regularizers.l2(reg_num)
                    ))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(drop_out))

    model.add(Conv2D(32,
                     kernel_size= (3,3), 
                     activation='relu',
                     kernel_regularizer=regularizers.l2(reg_num)
                    ))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(drop_out))
    
    #Flatten
    model.add(Flatten(input_shape=(image_height,image_width,3)))

    # Dense Layers
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(drop_out))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(drop_out))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(drop_out))
    
    # output layer
    model.add(Dense(num_classes, activation='softmax'))
    
    #Compile
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
    
    return model
    

In [21]:
nn = KerasClassifier(build_fn = run_nn, batch_size = 36, verbose=0)

In [22]:
params = {
    'epochs':[10,15,30],
    'layer_one_neurons':[32, 64, 255],
    'reg_num':[100,0,.001],
    'drop_out':[.01,.25,.35]
}

In [23]:
gs = GridSearchCV(nn, param_grid = params , cv = 5)

In [24]:
history = gs.fit(X_train, y_train)

Epoch 1/10
 - 3s - loss: 8610.7157 - acc: 0.0130
Epoch 2/10
 - 2s - loss: 6491.5966 - acc: 0.0130
Epoch 3/10
 - 2s - loss: 4805.8173 - acc: 0.0130
Epoch 4/10
 - 2s - loss: 3501.5797 - acc: 0.0130
Epoch 5/10
 - 2s - loss: 2514.6936 - acc: 0.0130
Epoch 6/10
 - 2s - loss: 1781.5272 - acc: 0.0130
Epoch 7/10
 - 2s - loss: 1245.4970 - acc: 0.0130
Epoch 8/10
 - 2s - loss: 859.2920 - acc: 0.0130
Epoch 9/10
 - 2s - loss: 584.9742 - acc: 0.0130
Epoch 10/10
 - 2s - loss: 392.9256 - acc: 0.0195
Epoch 1/10
 - 3s - loss: 8688.4416 - acc: 0.0097
Epoch 2/10
 - 2s - loss: 6587.9527 - acc: 0.0097
Epoch 3/10
 - 2s - loss: 4877.2123 - acc: 0.0097
Epoch 4/10
 - 2s - loss: 3556.7412 - acc: 0.0065
Epoch 5/10
 - 2s - loss: 2558.0537 - acc: 0.0227
Epoch 6/10
 - 2s - loss: 1815.5074 - acc: 0.0162
Epoch 7/10
 - 3s - loss: 1270.4678 - acc: 0.0129
Epoch 8/10
 - 2s - loss: 877.7061 - acc: 0.0162
Epoch 9/10
 - 2s - loss: 598.6205 - acc: 0.0129
Epoch 10/10
 - 2s - loss: 403.1030 - acc: 0.0129
Epoch 1/10
 - 3s - loss:

Epoch 5/10
 - 3s - loss: 2450.6107 - acc: 0.0162
Epoch 6/10
 - 3s - loss: 1676.3176 - acc: 0.0162
Epoch 7/10
 - 3s - loss: 1130.9788 - acc: 0.0162
Epoch 8/10
 - 3s - loss: 753.0729 - acc: 0.0162
Epoch 9/10
 - 3s - loss: 495.1755 - acc: 0.0162
Epoch 10/10
 - 3s - loss: 321.7829 - acc: 0.0162
Epoch 1/10
 - 4s - loss: 9576.3733 - acc: 0.0097
Epoch 2/10
 - 3s - loss: 7083.3405 - acc: 0.0097
Epoch 3/10
 - 3s - loss: 5080.2705 - acc: 0.0097
Epoch 4/10
 - 3s - loss: 3571.8740 - acc: 0.0097
Epoch 5/10
 - 3s - loss: 2470.4895 - acc: 0.0129
Epoch 6/10
 - 3s - loss: 1684.5498 - acc: 0.0097
Epoch 7/10
 - 3s - loss: 1133.2782 - acc: 0.0097
Epoch 8/10
 - 3s - loss: 752.6360 - acc: 0.0162
Epoch 9/10
 - 3s - loss: 493.8466 - acc: 0.0129
Epoch 10/10
 - 3s - loss: 320.4476 - acc: 0.0162
Epoch 1/10
 - 4s - loss: 9552.9921 - acc: 0.0065
Epoch 2/10
 - 3s - loss: 7070.6408 - acc: 0.0097
Epoch 3/10
 - 3s - loss: 5079.7444 - acc: 0.0129
Epoch 4/10
 - 3s - loss: 3580.2588 - acc: 0.0129
Epoch 5/10
 - 3s - loss:

 - 12s - loss: 851.5855 - acc: 0.0162
Epoch 8/10
 - 12s - loss: 582.1508 - acc: 0.0129
Epoch 9/10
 - 12s - loss: 395.5216 - acc: 0.0162
Epoch 10/10
 - 12s - loss: 265.9417 - acc: 0.0162
Epoch 1/10
 - 12s - loss: 4.9300 - acc: 0.0000e+00
Epoch 2/10
 - 12s - loss: 4.9240 - acc: 0.0032
Epoch 3/10
 - 12s - loss: 4.9120 - acc: 0.0130
Epoch 4/10
 - 12s - loss: 4.9014 - acc: 0.0130
Epoch 5/10
 - 13s - loss: 4.8802 - acc: 0.0162
Epoch 6/10
 - 12s - loss: 4.8547 - acc: 0.0227
Epoch 7/10
 - 12s - loss: 4.8217 - acc: 0.0162
Epoch 8/10
 - 12s - loss: 4.8065 - acc: 0.0357
Epoch 9/10
 - 12s - loss: 4.7884 - acc: 0.0195
Epoch 10/10
 - 12s - loss: 4.7351 - acc: 0.0260
Epoch 1/10
 - 13s - loss: 4.9329 - acc: 0.0032
Epoch 2/10
 - 12s - loss: 4.9195 - acc: 0.0227
Epoch 3/10
 - 12s - loss: 4.9037 - acc: 0.0097
Epoch 4/10
 - 12s - loss: 4.8739 - acc: 0.0129
Epoch 5/10
 - 12s - loss: 4.8469 - acc: 0.0065
Epoch 6/10
 - 12s - loss: 4.8291 - acc: 0.0097
Epoch 7/10
 - 12s - loss: 4.7859 - acc: 0.0227
Epoch 8/10

Epoch 9/15
 - 2s - loss: 574.1572 - acc: 0.0097
Epoch 10/15
 - 2s - loss: 385.1438 - acc: 0.0162
Epoch 11/15
 - 2s - loss: 254.9899 - acc: 0.0162
Epoch 12/15
 - 2s - loss: 166.7099 - acc: 0.0194
Epoch 13/15
 - 2s - loss: 107.6948 - acc: 0.0227
Epoch 14/15
 - 2s - loss: 69.0506 - acc: 0.0162
Epoch 15/15
 - 2s - loss: 44.2078 - acc: 0.0129
Epoch 1/15
 - 3s - loss: 4.9303 - acc: 0.0097
Epoch 2/15
 - 2s - loss: 4.9245 - acc: 0.0097
Epoch 3/15
 - 2s - loss: 4.9207 - acc: 0.0032
Epoch 4/15
 - 2s - loss: 4.9123 - acc: 0.0065
Epoch 5/15
 - 2s - loss: 4.8798 - acc: 0.0097
Epoch 6/15
 - 2s - loss: 4.8525 - acc: 0.0097
Epoch 7/15
 - 2s - loss: 4.8227 - acc: 0.0065
Epoch 8/15
 - 2s - loss: 4.7925 - acc: 0.0130
Epoch 9/15
 - 2s - loss: 4.7578 - acc: 0.0227
Epoch 10/15
 - 2s - loss: 4.7198 - acc: 0.0325
Epoch 11/15
 - 2s - loss: 4.6682 - acc: 0.0260
Epoch 12/15
 - 2s - loss: 4.5955 - acc: 0.0422
Epoch 13/15
 - 2s - loss: 4.5292 - acc: 0.0390
Epoch 14/15
 - 2s - loss: 4.4502 - acc: 0.0455
Epoch 15/15

Epoch 4/15
 - 3s - loss: 3479.0635 - acc: 0.0162
Epoch 5/15
 - 3s - loss: 2409.2358 - acc: 0.0162
Epoch 6/15
 - 3s - loss: 1644.0990 - acc: 0.0162
Epoch 7/15
 - 3s - loss: 1106.7349 - acc: 0.0194
Epoch 8/15
 - 3s - loss: 735.4335 - acc: 0.0162
Epoch 9/15
 - 3s - loss: 482.7517 - acc: 0.0162
Epoch 10/15
 - 3s - loss: 313.2953 - acc: 0.0162
Epoch 11/15
 - 3s - loss: 201.1910 - acc: 0.0162
Epoch 12/15
 - 3s - loss: 128.1515 - acc: 0.0194
Epoch 13/15
 - 3s - loss: 81.1594 - acc: 0.0162
Epoch 14/15
 - 3s - loss: 51.3292 - acc: 0.0129
Epoch 15/15
 - 3s - loss: 32.7283 - acc: 0.0194
Epoch 1/15
 - 4s - loss: 9457.3156 - acc: 0.0032
Epoch 2/15
 - 3s - loss: 6956.3069 - acc: 0.0032
Epoch 3/15
 - 3s - loss: 4994.0367 - acc: 0.0227
Epoch 4/15
 - 3s - loss: 3518.3918 - acc: 0.0129
Epoch 5/15
 - 3s - loss: 2441.7382 - acc: 0.0032
Epoch 6/15
 - 3s - loss: 1668.9534 - acc: 0.0000e+00
Epoch 7/15
 - 4s - loss: 1125.3459 - acc: 0.0097
Epoch 8/15
 - 3s - loss: 749.8928 - acc: 0.0129
Epoch 9/15
 - 3s - los

...

 - 3s - loss: 4.8904 - acc: 0.0032
Epoch 7/30
 - 3s - loss: 4.8755 - acc: 0.0227
Epoch 8/30
 - 3s - loss: 4.9039 - acc: 0.0097
Epoch 9/30
 - 3s - loss: 4.8825 - acc: 0.0194
Epoch 10/30
 - 3s - loss: 4.8512 - acc: 0.0097
Epoch 11/30
 - 3s - loss: 4.8831 - acc: 0.0000e+00
Epoch 12/30
 - 3s - loss: 4.8547 - acc: 0.0162
Epoch 13/30
 - 3s - loss: 4.8640 - acc: 0.0259
Epoch 14/30
 - 3s - loss: 4.8599 - acc: 0.0227
Epoch 15/30
 - 3s - loss: 4.8512 - acc: 0.0032
Epoch 16/30
 - 3s - loss: 4.8460 - acc: 0.0065
Epoch 17/30
 - 3s - loss: 4.8483 - acc: 0.0162
Epoch 18/30
 - 3s - loss: 4.8375 - acc: 0.0162
Epoch 19/30
 - 3s - loss: 4.8489 - acc: 0.0129
Epoch 20/30
 - 3s - loss: 4.8540 - acc: 0.0129
Epoch 21/30
 - 3s - loss: 4.8162 - acc: 0.0162
Epoch 22/30
 - 3s - loss: 4.8329 - acc: 0.0065
Epoch 23/30
 - 3s - loss: 4.8417 - acc: 0.0162
Epoch 24/30
 - 3s - loss: 4.8389 - acc: 0.0097
Epoch 25/30
 - 3s - loss: 4.8140 - acc: 0.0065
Epoch 26/30
 - 3s - loss: 4.8276 - acc: 0.0129
Epoch 27/30
 - 3s - loss

Epoch 2/30
 - 3s - loss: 5.0194 - acc: 0.0032
Epoch 3/30
 - 3s - loss: 5.0120 - acc: 0.0065
Epoch 4/30
 - 3s - loss: 4.9964 - acc: 0.0129
Epoch 5/30
 - 3s - loss: 4.9799 - acc: 0.0097
Epoch 6/30
 - 3s - loss: 4.9644 - acc: 0.0129
Epoch 7/30
 - 3s - loss: 4.9641 - acc: 0.0129
Epoch 8/30
 - 3s - loss: 4.9411 - acc: 0.0065
Epoch 9/30
 - 3s - loss: 4.9199 - acc: 0.0129
Epoch 10/30
 - 3s - loss: 4.9286 - acc: 0.0097
Epoch 11/30
 - 3s - loss: 4.9168 - acc: 0.0065
Epoch 12/30
 - 3s - loss: 4.9050 - acc: 0.0162
Epoch 13/30
 - 3s - loss: 4.9061 - acc: 0.0129
Epoch 14/30
 - 3s - loss: 4.8983 - acc: 0.0194
Epoch 15/30
 - 3s - loss: 4.9124 - acc: 0.0032
Epoch 16/30
 - 3s - loss: 4.8770 - acc: 0.0259
Epoch 17/30
 - 3s - loss: 4.8833 - acc: 0.0259
Epoch 18/30
 - 3s - loss: 4.8786 - acc: 0.0129
Epoch 19/30
 - 3s - loss: 4.8700 - acc: 0.0194
Epoch 20/30
 - 3s - loss: 4.8470 - acc: 0.0065
Epoch 21/30
 - 3s - loss: 4.8385 - acc: 0.0194
Epoch 22/30
 - 3s - loss: 4.8477 - acc: 0.0097
Epoch 23/30
 - 3s - l

Epoch 22/30
 - 12s - loss: 5.3516 - acc: 0.0129
Epoch 23/30
 - 12s - loss: 5.1154 - acc: 0.0129
Epoch 24/30
 - 12s - loss: 4.9486 - acc: 0.0356
Epoch 25/30
 - 12s - loss: 4.8784 - acc: 0.0162
Epoch 26/30
 - 12s - loss: 4.8691 - acc: 0.0162
Epoch 27/30
 - 12s - loss: 4.8292 - acc: 0.0162
Epoch 28/30
 - 12s - loss: 4.8258 - acc: 0.0194
Epoch 29/30
 - 12s - loss: 4.8255 - acc: 0.0194
Epoch 30/30
 - 12s - loss: 4.8134 - acc: 0.0129
Epoch 1/30
 - 13s - loss: 10446.3794 - acc: 0.0065
Epoch 2/30
 - 12s - loss: 6885.8287 - acc: 0.0097
Epoch 3/30
 - 12s - loss: 4418.7668 - acc: 0.0097
Epoch 4/30
 - 12s - loss: 2845.1216 - acc: 0.0129
Epoch 5/30
 - 12s - loss: 1865.5965 - acc: 0.0194
Epoch 6/30
 - 12s - loss: 1249.2157 - acc: 0.0065
Epoch 7/30
 - 12s - loss: 848.6225 - acc: 0.0097
Epoch 8/30
 - 12s - loss: 578.9121 - acc: 0.0097
Epoch 9/30
 - 12s - loss: 393.0725 - acc: 0.0097
Epoch 10/30
 - 12s - loss: 264.1734 - acc: 0.0194
Epoch 11/30
 - 12s - loss: 175.5269 - acc: 0.0129
Epoch 12/30
 - 12s -

Epoch 13/30
 - 12s - loss: 4.8154 - acc: 0.0065
Epoch 14/30
 - 12s - loss: 4.8018 - acc: 0.0065
Epoch 15/30
 - 12s - loss: 4.8344 - acc: 0.0129
Epoch 16/30
 - 13s - loss: 4.8165 - acc: 0.0162
Epoch 17/30
 - 17s - loss: 4.8331 - acc: 0.0097
Epoch 18/30
 - 18s - loss: 4.8172 - acc: 0.0129
Epoch 19/30
 - 12s - loss: 4.7872 - acc: 0.0194
Epoch 20/30
 - 12s - loss: 4.7945 - acc: 0.0097
Epoch 21/30
 - 12s - loss: 4.8134 - acc: 0.0162
Epoch 22/30
 - 12s - loss: 4.7724 - acc: 0.0065
Epoch 23/30
 - 12s - loss: 4.7920 - acc: 0.0065
Epoch 24/30
 - 12s - loss: 4.8050 - acc: 0.0065
Epoch 25/30
 - 12s - loss: 4.8091 - acc: 0.0065
Epoch 26/30
 - 12s - loss: 4.7704 - acc: 0.0129
Epoch 27/30
 - 12s - loss: 4.7822 - acc: 0.0162
Epoch 28/30
 - 12s - loss: 4.7777 - acc: 0.0194
Epoch 29/30
 - 12s - loss: 4.7825 - acc: 0.0129
Epoch 30/30
 - 12s - loss: 4.7644 - acc: 0.0259
Epoch 1/30
 - 13s - loss: 5.0498 - acc: 0.0097
Epoch 2/30
 - 12s - loss: 5.0199 - acc: 0.0162
Epoch 3/30
 - 12s - loss: 5.0095 - acc: 0.

Epoch 5/30
 - 4s - loss: 4.9295 - acc: 0.0052
Epoch 6/30
 - 4s - loss: 4.9110 - acc: 0.0130
Epoch 7/30
 - 4s - loss: 4.8980 - acc: 0.0181
Epoch 8/30
 - 4s - loss: 4.8846 - acc: 0.0130
Epoch 9/30
 - 5s - loss: 4.8862 - acc: 0.0130
Epoch 10/30
 - 6s - loss: 4.8653 - acc: 0.0078
Epoch 11/30
 - 5s - loss: 4.8529 - acc: 0.0130
Epoch 12/30
 - 5s - loss: 4.8394 - acc: 0.0155
Epoch 13/30
 - 5s - loss: 4.8354 - acc: 0.0259
Epoch 14/30
 - 4s - loss: 4.8113 - acc: 0.0337
Epoch 15/30
 - 4s - loss: 4.7797 - acc: 0.0155
Epoch 16/30
 - 4s - loss: 4.7256 - acc: 0.0259
Epoch 17/30
 - 4s - loss: 4.6432 - acc: 0.0389
Epoch 18/30
 - 4s - loss: 4.5353 - acc: 0.0466
Epoch 19/30
 - 4s - loss: 4.3873 - acc: 0.0674
Epoch 20/30
 - 4s - loss: 4.2935 - acc: 0.0648
Epoch 21/30
 - 5s - loss: 4.1549 - acc: 0.0777
Epoch 22/30
 - 4s - loss: 3.9149 - acc: 0.0907
Epoch 23/30
 - 4s - loss: 3.6749 - acc: 0.1606
Epoch 24/30
 - 4s - loss: 3.3711 - acc: 0.1684
Epoch 25/30
 - 4s - loss: 3.2169 - acc: 0.2306
Epoch 26/30
 - 4s 

In [25]:
print(gs.best_score_) # accuracy

0.025874126330018045


In [26]:
gs.best_params_

{'drop_out': 0.01, 'epochs': 30, 'layer_one_neurons': 64, 'reg_num': 0.001}

In [45]:
joblib.dump(history, "History/Species_pickle.pkl")

['History/Species_pickle.pkl']

# Test predictions

In [50]:
species_list = sorted(os.listdir(f'{root_folder}/test'))


try:
    species_list.remove('.DS_Store')
    species_list.remove('.ipynb_checkpoints')
except:
    print('file not in folder')

file not in folder


In [51]:
species_list[:5]

['agaricus_arvensis',
 'agaricus_augustus',
 'albatrellus_confluens',
 'albatrellus_ovinus',
 'amanita_ceciliae']

In [52]:
len(species_list)

139

In [53]:
sorted(os.listdir(f'{root_folder}/test/agaricus_arvensis'))

['agaricusarvensis2.jpg']

In [54]:
test_df.insert(0, 'class', range(0, len(test_df)))


In [55]:
test_df.head()

,class,image_path
0,0,agaricus_arvensis/agaricusarvensis2.jpg
1,1,agaricus_augustus/agaricusaugustus2.jpg
2,2,albatrellus_confluens/albatrellusconfluens2.jpg
3,3,amanita_fulva/amanitafulva2.jpg
4,4,amanita_muscaria/amanitamuscaria2.jpg


In [56]:
test_image = []
count=0
while count <len(test_df):
    img = image.load_img(f"{root_folder}/test/{test_df.loc[count]['image_path']}", 
                         target_size=(image_height,image_width,3), color_mode='rgb')
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
    count+=1


In [57]:
test_array = np.array(test_image)

In [61]:
predictions = gs.predict(test_array)

In [62]:
predictions

array([  9,  26,  97,  58,  68,   6, 137,  43,  71, 121,   9,  37,  30,
        84, 119, 120, 132,  47,  30,  83,  57,  71, 132,   6,  58,  13,
        38,  94,  92,  33,  13,  89,  36, 108, 100,  55,  97,   7,  50,
       112,  42, 118,  50,  38, 105,  63,  72,  57,  20,  78,  71,  85,
        50, 108,   8,  22,  87,  80,  81,  92, 110, 111,  76, 125,  80,
        84, 105,  86,  72,  80,  58,  17, 131, 119,  53, 134,  84,  81,
       103, 127,  78,  95, 103,  28, 123,  87, 137, 114,  37, 137,  42,
        58,  89,   3,  72,  72,  76, 100,  76,  72,  28,   6,  47, 103,
        38,  92,  63, 129, 133,   6, 102,  78,  81, 128,  30, 129,  97,
        87,  89,  72,  95,  35,   0, 129,  64, 116,  72, 119,  97,  27,
       129, 121,  59,   9, 117])

In [63]:
len(predictions)

135

In [64]:
test_df['predictions'] = predictions

In [65]:
test_df.tail()

,class,image_path,predictions
130,130,tricholoma_sejunctum/tricholomasejunctum2.jpg,129
131,131,tricholomopsis_decora/tricholomopsisdecora2.jpg,121
132,132,tricholomopsis_rutilans/tricholomopsisrutilans...,59
133,133,turbinellus_floccosus/turbinellusfloccosus2.jpg,9
134,134,tylopilus_felleus/tylopilusfelleus2.jpg,117


In [66]:
(num_correct = len(test_df.loc[test_df["class"] != predictions]) - len(test_df)+1)

In [67]:
print (f"{num_correct} total correct predictions")

0 total correct predictions


In [68]:
test_df.to_csv("CSVs/preds_species.csv", index=False)